# Recreate Figure 2H

In [ ]:
library(fgsea)
library(extrafont)
library(cowplot)
library(ggpubr)
library(ggrepel)
library(dplyr)
library(gplots)
library(RColorBrewer)
library(tibble)

In [ ]:
## Import differentially expressed gene lists
mono_bvc_f <- read.table("Aug20_BLZ_data/Mono-CSF1Ri_vs_Mono-Vehicle_fdr05_Aug20.csv", sep="\t", header=F)
mono_bvc_f <- deframe(mono_bvc_f)
mac_bvc_f <- read.table("Aug20_BLZ_data/Mac-CSF1Ri_vs_Mac-Vehicle_fdr05_Aug20.csv", sep="\t", header=F)
mac_bvc_f <- deframe(mac_bvc_f)

In [ ]:
## Import all GO BP pathways and genes
gofile <- scan("GOids_genes.txt", what="", sep="\n")   
pathways_all <- strsplit(gofile, "\t")

# Extract the first vector element and set it as the list element name
names(pathways_all) <- sapply(pathways_all, `[[`, 1)
# Remove the first vector element from each list element
pathways_all <- lapply(pathways_all, `[`, -1)

In [ ]:
# fgsea - only retain pathways where 15+ genes expressed and no more than 300
set.seed(1234)
mono_res <- fgsea(pathways=pathways_all, stats=mono_bvc_f, nperm=10000, minSize=15, maxSize=300)
mac_res <- fgsea(pathways=pathways_all, stats=mac_bvc_f, nperm=10000, minSize=15, maxSize=300)

In [ ]:
# write all results to file
mono_f <- mono_res
mono_f$leadingEdge <- vapply(mono_f$leadingEdge, paste, collapse = ", ", character(1L))
mac_f <- mac_res
mac_f$leadingEdge <- vapply(mac_f$leadingEdge, paste, collapse = ", ", character(1L))

write.table(mono_f, file="Aug20_BLZ_data/monocyte_fgsea_results_full_Aug20.txt",
            sep="\t", col.names=T, row.names=F, quote=F)
write.table(mac_f, file="Aug20_BLZ_data/macrophage_fgsea_results_full_Aug20.txt",
            sep="\t", col.names=T, row.names=F, quote=F)

In [ ]:
# label pathways a/w immune activation
mono_plot <- mono_res[ES>0][order(-NES)][padj<0.05,]
mono_plot$fn <- "no"

mono_ias <- read.delim("Aug20_BLZ_data/mono_imm-act-list_Aug20.txt", header=F)$V1
mono_plot$fn[(mono_plot$pathway %in% mono_ias)] <- "immune activation"

In [ ]:
mac_plot <- mac_res[ES>0][order(-NES)][padj<0.05,]
mac_plot$fn <- "no"

mac_ias <- read.delim("Aug20_BLZ_data/mac_imm-act-list_Aug20.txt", header=F)$V1
mac_plot$fn[(mac_plot$pathway %in% mac_ias)] <- "immune activation"